In [ ]:
import numpy as np
import pandas as pd
import glob
import h5py
import hls4ml
import plotting
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from qkeras import QConv2D, QDense, QActivation
import tensorflow_model_optimization as tfmot
from tensorflow.keras.layers import (
    Lambda,
    Input,
    Dense,
    LeakyReLU,
    Conv2D,
    AveragePooling2D,
    MaxPooling2D,
    UpSampling2D,
    ZeroPadding2D,
    BatchNormalization,
    Flatten,
    Reshape,
    Activation,
    Concatenate,
    Cropping1D
)

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def create_dataset(location):
    
    # retrieve dataset from h5 file
    h5file = h5py.File(location, "r")
    data = h5file["jetConstituentsList"][()]
    features = h5file["particleFeatureNames"][()]
    target = np.copy(data)

    h5file.close()

    # split the data and target data
    X_train, X_test, Y_train, Y_test = train_test_split(data, target, test_size=0.5, shuffle=True)

    # reshpae the dataset
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
    Y_train = Y_train.reshape(Y_train.shape[0], Y_train.shape[1], Y_train.shape[2], 1)
    Y_test = Y_test.reshape(Y_test.shape[0], Y_test.shape[1], Y_test.shape[2], 1)

    return X_train, X_test, Y_train, Y_test

def create_conv_ae(latent_dim=8, quant_size=0, pruning=False):

    # encoder
    input_encoder = Input(shape=(16,3,1), name='encoder_input')
    x = BatchNormalization()(input_encoder)
    x = Conv2D(16, kernel_size=(3,3), use_bias=False, padding='valid')(x) if quant_size==0 \
        else QConv2D(16, kernel_size=(3,3), use_bias=False, padding='valid',
                         kernel_quantizer='quantized_bits(' + str(quant_size) + ',0,1)')(x)
    x = Activation('relu')(x) if quant_size==0 \
        else QActivation('quantized_relu(bits=' + str(quant_size) + ')')(x)
    x = AveragePooling2D(pool_size=(3, 1))(x)
    x = Conv2D(32, kernel_size=(3,1), use_bias=False, padding='same')(x) if quant_size==0 \
        else QConv2D(32, kernel_size=(3,1), use_bias=False, padding='same',
                         kernel_quantizer='quantized_bits(' + str(quant_size) + ',0,1)')(x)
    x = Activation('relu')(x) if quant_size==0 \
        else QActivation('quantized_relu(bits=' + str(quant_size) + ')')(x)
    x = AveragePooling2D(pool_size=(3, 1))(x)
    x = Flatten()(x)

    enc = Dense(latent_dim)(x)
    encoder = Model(inputs=input_encoder, outputs=enc)
    encoder.summary()

    # decoder
    input_decoder = Input(shape=(latent_dim,), name='decoder_input')
    x = Dense(32)(input_decoder) if quant_size==0 \
        else QDense(32, kernel_quantizer='quantized_bits(' + str(quant_size) + ',2,1)',
                    bias_quantizer='quantized_bits(' + str(quant_size) + ',2,1)')(input_decoder)
    x = Activation('relu')(x) if quant_size==0 \
        else QActivation('quantized_relu(bits=' + str(quant_size) + ')')(x)
    x = Reshape((1,1,32))(x)
    x = UpSampling2D((4,1))(x)
    x = Conv2D(32, kernel_size=(3,1), use_bias=False, padding='same')(x) if quant_size==0 \
        else QConv2D(32, kernel_size=(3,1), use_bias=False, padding='same',
                         kernel_quantizer='quantized_bits(' + str(quant_size) + ',0,1)')(x)
    x = Activation('relu')(x) if quant_size==0 \
        else QActivation('quantized_relu(bits=' + str(quant_size) + ')')(x)
    x = Conv2D(16, kernel_size=(3,1), use_bias=False, padding='same')(x) if quant_size==0 \
        else QConv2D(16, kernel_size=(3,3), use_bias=False, padding='same',
                         kernel_quantizer='quantized_bits(' + str(quant_size) + ',0,1)')(x)
    x = Activation('relu')(x) if quant_size==0 \
        else QActivation('quantized_relu(bits=' + str(quant_size) + ')')(x)
    x = UpSampling2D((4,3))(x)

    dec = Conv2D(1, kernel_size=(3,3), use_bias=False, padding='same')(x) if quant_size==0 \
        else QConv2D(1, kernel_size=(3,3), use_bias=False, padding='same',
                        kernel_quantizer='quantized_bits(' + str(quant_size) + ',0,1)')(x)
    decoder = Model(inputs=input_decoder, outputs=dec)
    decoder.summary() # AE
    ae_outputs = decoder(encoder(input_encoder))
    autoencoder = Model(inputs=input_encoder, outputs=ae_outputs)
    autoencoder.summary()

    if pruning:
        pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
                                initial_sparsity=0.0, final_sparsity=0.5,
                                begin_step=0, end_step=40000)
        encoder_pruned = tfmot.sparsity.keras.prune_low_magnitude(encoder, pruning_schedule=pruning_schedule)
        encoder = encoder_pruned
        decoder_pruned = tfmot.sparsity.keras.prune_low_magnitude(decoder, pruning_schedule=pruning_schedule)
        decoder = decoder_pruned

    # compile AE
    autoencoder.compile(optimizer=Adam(lr=3E-3, amsgrad=True), loss='mse')
    return autoencoder, encoder

In [ ]:
# ----------------------------------
# Train - WIP!
# ----------------------------------

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# GPU config
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

# load dataset and split into test and train files
X_train, X_test, Y_train, Y_test = create_dataset("../../data/bkg_3mln.h5")

# define callbacks
callbacks=[
    EarlyStopping(monitor='val_loss', patience=10, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1),
    # tfmot.sparsity.keras.UpdatePruningStep() # pruning
]

# get the CNN autoencoder
model, encoder = create_conv_ae(quant_size=0, pruning=False)

# begin training
batch_size = 1024
n_epochs = 20

hist = model.fit(
        x=X_train,
        y=Y_train,
        epochs=n_epochs,
        batch_size=batch_size,
        verbose=2,
        validation_split=0.2,
        callbacks=callbacks)

pred = model.predict(X_test)
encoded = encoder.predict(X_test)

config = hls4ml.utils.config_from_keras_model(model, granularity='model')
print("-----------------------------------")
print("Configuration")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       output_dir='model_1/hls4ml_prj',
                                                       fpga_part='xcu250-figd2104-2L-e')

hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)
hls_model.compile()
X_test = np.ascontiguousarray(X_test)
y_hls = hls_model.predict(X_test)

print("Keras  Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(pred, axis=1))))
print("hls4ml Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_hls, axis=1))))

fig, ax = plt.subplots(figsize=(9, 9))
_ = plotting.makeRoc(Y_test, pred, le.classes_)
plt.gca().set_prop_cycle(None) # reset the colors
_ = plotting.makeRoc(Y_test, y_hls, le.classes_, linestyle='--')

from matplotlib.lines import Line2D
lines = [Line2D([0], [0], ls='-'),
         Line2D([0], [0], ls='--')]
from matplotlib.legend import Legend
leg = Legend(ax, lines, labels=['keras', 'hls4ml'],
            loc='lower right', frameon=False)
ax.add_artist(leg)